In [142]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
import seaborn as sns
import os
import plotly.io as pio
pio.renderers.default = "notebook"
import plotly.express as px

%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Preprocesamiento

In [143]:
def read_text_file(file_name):
    with open(file_name, "r") as f:
        return f.read()


def get_line_number_with_string(file_name, string):
    with open(file_name, "r") as f:
        for i, line in enumerate(f):
            if string in line:
                return i


def select_line_number(raw, line_nr):
    return raw.split("\n")[line_nr]

In [144]:
log_file_path = "log.log"

In [145]:
periodos_modelo = int(
    read_text_file("model_inputs/parametros.dat").split("\n")[1].split(" ")[1]
)
semanas_modelo = int(
    read_text_file("model_inputs/parametros.dat").split("\n")[2].split(" ")[1]
)

In [146]:
Log_df = pd.DataFrame()

raw = read_text_file(log_file_path)
results_line_nr = get_line_number_with_string(
    log_file_path, "================================="
)


# once per log
objective_value = select_line_number(raw, results_line_nr + 2).split(" ")[-1]


row_from = (
    get_line_number_with_string(log_file_path, "=================================") + 3
)
row_to = get_line_number_with_string(log_file_path, "Statistics") - 2

# multiple times per log
for line_nr in range(row_from, row_to + 1):
    variable = select_line_number(raw, line_nr).split(" ")[0]
    value = select_line_number(raw, line_nr).split(" ")[-2]
    unit_impact_on_obj_func = select_line_number(raw, line_nr).split(" ")[-1][6:-1]
    row_data = pd.Series(
        {
            "variable": variable,
            "value": value,
            "unit_impact_on_obj_func": unit_impact_on_obj_func,
        }
    )
    Log_df = pd.concat([Log_df, pd.DataFrame([row_data])], ignore_index=True)

print(objective_value)

-8589439


In [147]:
Log_df["var"] = Log_df["variable"].str.split("#").str[0]
Log_df["t"] = Log_df["variable"].str.split("#").str[1]
Log_df["age"] = Log_df["variable"].str.split("#").str[2]
Log_df["class"] = Log_df["variable"].str.split("#").str[3]

In [148]:
Log_df = Log_df[["t", "var", "age", "class", "value", "unit_impact_on_obj_func"]]
Log_df["t"] = Log_df["t"].astype(int)
Log_df["age"] = Log_df["age"].astype(int)
Log_df["value"] = Log_df["value"].astype(float)
Log_df["unit_impact_on_obj_func"] = Log_df["unit_impact_on_obj_func"].astype(float)
Log_df["impact_on_obj_func"] = Log_df["value"] * Log_df["unit_impact_on_obj_func"]
Log_df["class"] = Log_df["class"].astype(str)

# agregamos artificialmente filas de venta 0 en t0 (igulamente estan restringidas) para la correcta visualizacion simetrica de los barplots
Log_df = pd.concat(
    [
        Log_df,
        pd.DataFrame(
            [
                pd.Series(
                    {
                        "t": 0,
                        "var": "y",
                        "age": 0,
                        "class": "1",
                        "value": 0,
                        "unit_impact_on_obj_func": 0.0,
                        "impact_on_obj_func": 0,
                    }
                )
            ]
        ),
    ],
    ignore_index=True,
)
Log_df = pd.concat(
    [
        Log_df,
        pd.DataFrame(
            [
                pd.Series(
                    {
                        "t": 0,
                        "var": "y",
                        "age": 0,
                        "class": "2",
                        "value": 0,
                        "unit_impact_on_obj_func": 0.0,
                        "impact_on_obj_func": 0,
                    }
                )
            ]
        ),
    ],
    ignore_index=True,
)
Log_df = pd.concat(
    [
        Log_df,
        pd.DataFrame(
            [
                pd.Series(
                    {
                        "t": 0,
                        "var": "y",
                        "age": 0,
                        "class": "3",
                        "value": 0,
                        "unit_impact_on_obj_func": 0.0,
                        "impact_on_obj_func": 0,
                    }
                )
            ]
        ),
    ],
    ignore_index=True,
)
Log_df.sort_values(by="class", inplace=True)

# Comportamiento variables  
stock
venta
nacimientos
traspasos

In [154]:
# maximos/minimos para rangos del eje Y
x_max = Log_df.loc[Log_df["var"] == "x"].value.max()
x_min = Log_df.loc[Log_df["var"] == "x"].value.min()
y_max = Log_df.loc[Log_df["var"] == "y"].value.max()
y_min = Log_df.loc[Log_df["var"] == "y"].value.min()

stock = Log_df.loc[(Log_df["var"] == "x")]
fig1 = px.bar(
    stock,
    x="t",
    y="value",
    color="age",
    color_continuous_scale="brwnyl",
    facet_col="class",
    title="stock por edad y clase",
    range_x=[0, periodos_modelo],
    range_y=[0, x_max],
)
fig1.update_traces(width=1)
fig1.update_layout(
    xaxis_title="tiempo",
    xaxis2_title="tiempo",
    xaxis3_title="tiempo",
    yaxis_title="cantidad",
)
fig1.show()

ventas = Log_df.loc[(Log_df["var"] == "y")]

fig2 = px.bar(
    ventas,
    x="t",
    y="value",
    color="age",
    color_continuous_scale="Blugrn",
    facet_col="class",
    title="ventas por edad y clase",
    range_x=[0, periodos_modelo],
    range_y=[0, y_max],
)
fig2.update_traces(width=2)
fig2.update_layout(
    xaxis_title="tiempo",
    xaxis2_title="tiempo",
    xaxis3_title="tiempo",
    yaxis_title="cantidad",
)
fig2.show()

trasp_df = Log_df.loc[Log_df["var"] == "w"].sort_values(by="t")
nacimientos_df = (
    Log_df.loc[Log_df["var"] == "n"].sort_values(by="t").drop("class", axis=1)
)
nacimientos_df = nacimientos_df.rename(columns={"age": "class"})
nacimientos_df["class"] = nacimientos_df["class"].astype(str)

try:
    # nacimientos barplot
    fig4 = px.bar(
        x=nacimientos_df.t,
        y=nacimientos_df.value,
        range_x=[-5, periodos_modelo],
        color=nacimientos_df["class"],
        title="nacimientos por clase",
        color_discrete_sequence=px.colors.qualitative.T10,
    )
    fig4.update_traces(width=2)
    fig4.update_layout(
        height=500, width=600, xaxis_title="tiempo", yaxis_title="cantidad"
    )
    fig4.show()
except ValueError:
    pass

try:
    # traspasos barplot
    fig3 = px.bar(
        x=trasp_df.t,
        y=trasp_df.value,
        range_x=[-5, periodos_modelo],
        title="traspasos de clase 2 a 3",
        color_discrete_sequence=px.colors.qualitative.Vivid,
    )
    fig3.update_traces(width=2)
    fig3.update_layout(
        height=500, width=600, xaxis_title="tiempo", yaxis_title="cantidad"
    )
    fig3.show()
except ValueError:
    pass

In [ ]:
Log_df.loc[Log_df["var"] == "x"].loc[Log_df["class"] == "1"].loc[Log_df["t"] == 14]

In [ ]:
df_precios = pd.read_csv("precios.csv")
df_precios["PERIODO_INICIO"] = pd.to_datetime(df_precios["PERIODO_INICIO"])
df_precios_plot = df_precios.loc[
    (df_precios["PERIODO_INICIO"] > pd.to_datetime(precio_refe["fecha_min"]))
    & (df_precios["PERIODO_INICIO"] < pd.to_datetime(precio_refe["fecha_max"]))
]
df_precios_plot = df_precios_plot.melt(
    id_vars=["PERIODO_INICIO"],
    value_vars=["VAQUILLONAS270", "VAQUILLONAS391", "NOVILLITOS300", "NOVILLITOS391"],
)
fig = px.line(
    df_precios_plot,
    x="PERIODO_INICIO",
    y="value",
    color="variable",
    title="precios de referencia",
)

# Comportamiento funcion objetivo

In [ ]:
group_obj_func = Log_df.groupby(["var", "t"])["impact_on_obj_func"].sum().reset_index()

# total value per period
obj_sum = (
    group_obj_func.groupby("t")
    .sum()
    .rename(columns={"impact_on_obj_func": "impact_on_obj_func"})
)
obj_sum["var"] = "total"
obj_sum.reset_index(inplace=True)

# CUIDADO, AGREGO EL TOTAL COMO UNA VARIABLE MAS, NO USAR PARA OTRAS COSAS
# PORQUE TIENE INFORMACION DUPLICADA
group_obj_func_2 = pd.concat([group_obj_func, obj_sum], axis=0)
group_obj_func_2 = group_obj_func_2.loc[group_obj_func_2.impact_on_obj_func != 0]
fig = px.scatter(
    x=group_obj_func_2.t,
    y=group_obj_func_2.impact_on_obj_func,
    color=group_obj_func_2["var"],
    range_x=[-5, periodos_modelo],
    title="variacion de funcion de ganancia",
)
fig.update_layout(xaxis_title="tiempo", yaxis_title="impacto funcion objetivo")
fig.show()

cumsum_df = group_obj_func.groupby("t").sum().reset_index()
cumsum_df["cumsum_obj_func"] = cumsum_df["impact_on_obj_func"].cumsum()
fig2 = px.line(
    cumsum_df, x="t", y="cumsum_obj_func", title="ganancia acumulada en el tiempo"
)
fig2.update_layout(xaxis_title="tiempo", yaxis_title="funcion objetivo acumulada")
fig2.show()